In [1]:
import pandas as pd

In [5]:
df_overview = pd.read_csv("./Data/processed/LongForm_Clean_Lemma_Telehealth.csv")

In [6]:
from rake_nltk import Rake

keyword_corpus = " ".join(df_overview["LongForm_Clean_Content_Lemma"].str.replace("et al",""))

r_extraction = Rake()

r_extraction.extract_keywords_from_text(keyword_corpus)


r_extraction.get_ranked_phrases()[:10]

['treat various disorder bee 2008 postel de haan de jong 2008 include posttraumatic stress disorder germain marchand bouchard drouin guay 2009 depression sloan gallagher feinstein lee pruneau 2011 anxiety ruskin 2004 substance use frueh henderson myrick 2005 chronic pain macea gajos calil fregni 2010',
 'largely positive adler pritchett kauth nadorff 2014 baird whitney caedo 2018 brooks manson bair dailey shore 2012 cunningham connors lever stephan 2013 levy strachan 2013 mitchell maclaren mortimes locate withinn carachi 2009 moreau 2018 whitten kuwahara 2004 wynn bergvik pettersen fossum 2012',
 'neuropsychological testing bouchard 2004 cullum hynan grosch parikh weiner 2014 cullum weiner gehrmasculine gendern hynan 2006 gehrmasculine gender shah miles kuna godleski 2016 gros yoder tuerk lozano acierno 2011 hilty 2013 morland 2014 morland hynes mackintimes locate withinsh resick chard 2011the veteran ’',
 'autism goodwin mazefsky ioannidis erdogmus siegel 2019 hollis falconer martin w